# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043338,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047277,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.137244,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.150212,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.220703,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
base_clf = MLPClassifier()
base_clf.fit(X_train, y_train)

def eval_model(dataset_train, dataset_test, use_base_clf=True):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    if use_base_clf:
        y_train = base_clf.predict(X_train)

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    # dp = DP(mlp, X_test)
    # ftu = FTU(X_train, y_train, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test, use_base_clf=False)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8893129770992366,
 'recall': 0.9307589880159787,
 'auroc': 0.7906806988272664}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-01-26 14:25:53.090096: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-26 14:25:53.126442: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Synthetic data generation: 100%|██████████| 221/221 [00:02<00:00, 105.63it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.073302,0.166667,0.035610,0.000000,0.466667,0.166667,0.615385,0.4,0.75,1.0,-0.026500,-0.010216,0.072761,0.350,0.0
1,0.073536,0.500000,0.041512,0.266667,0.866667,0.000000,1.000000,0.6,0.50,0.0,-0.035191,-0.005636,0.075068,0.275,1.0
2,0.071980,0.000000,0.042045,0.733333,0.000000,0.833333,0.307692,0.6,0.00,0.0,-0.033259,-0.006952,0.069110,0.425,1.0
3,0.077395,0.000000,0.042382,0.866667,0.133333,0.166667,0.692308,0.4,1.00,1.0,-0.038062,-0.005608,0.077118,0.250,1.0
4,0.074207,0.166667,0.043648,0.133333,0.666667,0.500000,0.230769,0.6,0.25,0.0,-0.037334,-0.005840,0.078008,0.750,1.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 57/57 [00:00<00:00, 79.82it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.514540,0.333333,-0.188157,0.666667,0.200000,0.833333,0.307692,0.0,1.00,0.0,-0.017611,0.159389,0.774344,0.800,0.0
1,-0.427536,0.833333,-0.273311,0.600000,0.400000,0.833333,0.769231,0.0,0.75,0.0,-0.020703,0.248819,0.710468,0.625,1.0
2,-0.452164,0.833333,-0.259723,0.333333,0.533333,0.000000,1.000000,0.6,0.75,1.0,-0.032020,0.218813,0.725078,0.400,1.0
3,-0.481865,0.666667,-0.225188,1.000000,0.333333,1.000000,0.076923,1.0,1.00,1.0,-0.018525,0.133929,0.727700,0.875,1.0
4,-0.363084,0.500000,-0.163527,0.066667,0.666667,0.000000,0.615385,1.0,0.75,1.0,0.006239,0.187622,0.624344,1.000,1.0


In [12]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

INFO:tensorflow:Restoring parameters from fair-9
burning in


2022-01-26 14:26:00.218463: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


generating


/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.895464,0.000000,0.308918,0.067286,0.000000,0.131079,0.000000,0.278640,1.752207,0.0,0.0,0.423585,0.0,0.471339,1.0
1,0.000000,0.104030,0.000000,0.645687,0.454629,0.018567,0.588073,0.000000,0.085750,0.0,0.0,0.491787,0.0,0.623588,1.0
2,0.027066,0.213806,0.497826,0.557692,0.587919,0.596714,0.858378,0.000000,0.621208,0.0,0.0,0.383516,0.0,1.051334,1.0
3,0.000000,0.000000,0.000000,0.383852,0.000000,0.092355,0.440810,0.000000,0.943324,0.0,0.0,0.595602,0.0,0.000000,1.0
4,0.000000,0.091245,0.480436,0.466834,0.087119,0.179120,0.347991,0.154661,0.236591,0.0,0.0,0.591424,0.0,1.015640,1.0


In [14]:
eval_model(synth_data, dataset_test, use_base_clf=False)

{'precision': 0.7787846481876333,
 'recall': 0.9727030625832224,
 'auroc': 0.5696848646249445}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
820,0.127986,6.505250e-03,0.260719,0.545312,0.508612,0.017305,0.452107,0.381281,1.720641e-17,1.000000,0.014000,0.001290,0.555774,1.189650e-04,0.0
8090,0.131818,9.212060e-09,0.185745,0.200896,0.532807,0.018446,0.174596,0.324517,2.845341e-02,0.999825,0.001835,0.070755,0.546974,3.457324e-09,1.0
21720,0.058111,1.249064e-08,0.092637,0.183181,0.497256,0.477349,0.096520,0.418476,1.436928e-14,0.001563,0.003901,0.009430,0.199226,2.038802e-04,0.0
8533,0.097058,6.320673e-01,0.110523,0.109445,0.491563,0.003101,0.277898,0.401286,2.051231e-09,1.000000,0.002185,0.012075,0.556059,1.106022e-09,1.0
384,0.079833,8.544105e-01,0.275441,0.079588,0.813641,0.478263,0.079495,0.092369,7.810325e-03,0.001496,0.001527,0.010691,0.164217,4.920097e-05,1.0


In [16]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8633353401565322,
 'recall': 0.9547270306258322,
 'auroc': 0.7494518687265707}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
820,0.153768,3.715958e-02,0.055251,0.024270,0.772257,0.084455,0.279676,0.378635,5.725345e-09,0.999998,0.011492,0.000643,0.554751,2.311207e-02,0.0
8090,0.396614,8.847225e-01,0.136195,0.313493,0.526973,0.478461,0.441758,0.079517,4.294725e-12,0.002494,0.009005,0.005971,0.537635,2.288050e-05,0.0
21720,0.092177,4.620065e-06,0.030696,0.037403,0.748397,0.336347,0.444528,0.124008,3.347828e-14,1.000000,0.002300,0.019050,0.506784,3.116096e-06,0.0
8533,0.246952,1.120284e-08,0.238834,0.080244,0.535687,0.244846,0.211357,0.539743,3.568396e-10,0.001391,0.004309,0.000482,0.546833,2.517349e-07,0.0
384,0.047278,1.017363e-04,0.110711,0.020482,0.819452,0.480659,0.098762,0.444936,4.951538e-09,0.453703,0.004009,0.000101,0.143042,8.183905e-05,0.0


In [18]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8665458937198067,
 'recall': 0.9553928095872171,
 'auroc': 0.7558088545928053}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
820,0.425065,0.000016,0.076167,0.036175,0.525446,0.311616,0.567957,0.513070,9.933890e-01,1.000000,0.020585,0.004924,0.551238,1.949125e-07,1.0
8090,0.204001,0.038058,0.048925,0.287193,0.802236,0.478439,0.149760,0.914703,1.423087e-07,0.001585,0.017350,0.000417,0.216526,4.871781e-02,1.0
21720,0.532660,0.036667,0.263847,0.672467,0.602407,0.003868,0.092006,0.634670,1.350909e-15,0.996266,0.013391,0.000071,0.335395,3.130129e-01,0.0
8533,0.217943,0.003277,0.081022,0.576501,0.799014,0.065300,0.190752,0.057998,3.050571e-04,0.001273,0.001926,0.001728,0.442880,5.171908e-04,1.0
384,0.533648,0.000950,0.116004,0.108454,0.491077,0.048409,0.717474,0.131505,9.935881e-01,1.000000,0.006858,0.008056,0.529465,1.277113e-07,0.0


In [20]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8704379562043796,
 'recall': 0.9527296937416777,
 'auroc': 0.7625094251840918}